# Introduction

The `tiledbvcf` Python module allows you to create, update, and query TileDB-VCF datasets. It was designed to facilitate convenient analysis of genomic variant data through integration with Pandas. It also provides convenient interfaces for batch-processing queries and natively supports remote remote data stores like S3. 

*Hint: By the way, you can launch an interactive version of this tutoral using TileDB Cloud Notebooks. Simply start a session with the Genomics & Geospatial image and use the integrated file browser to open the notebook: examples/genomics/02-vcf-python.ipynb.*


# Setup

This tutorial requires the following modules.

In [ ]:
import tiledbvcf

import os
import tempfile
import glob
import pandas as pd

# Storing VCF files in TileDB

Similar to TileDB-VCF's command-line interface (CLI), `tiledbvcf` supports ingesting VCF (or BCF) files into TileDB, either when creating a new dataset *or* updating an existing datset with additional samples. See the [CLI tutorial](https://docs.tiledb.com/genomics/usage/cli) for a more detailed description of the ingestion process. Here, we'll only focus on the mechanics of ingestion from Python.

## Local VCFs

The `data/vcf` directory contains sample-level `vcf` files for the first 3 individuals from a synthetic dataset.

In [ ]:
local_vcfs = glob.glob("data/vcfs/*vcf.gz")
local_vcfs

Before you can ingest these files into TileDB an empty **TileDB VCF dataset** must first be created and opened in *write* mode.

In [ ]:
small_ds = tiledbvcf.Dataset('data/small_dataset2', mode = "w")
small_ds

This returns a new `Dataset` object, into which the VCF files can be ingested and stored.

In [ ]:
small_ds.ingest_samples(local_vcfs)

You can now re-open the dataset in *read* mode and begin to query it. For example, use `count()` to return the total number of ranges, across all samples contained within the dataset.

In [ ]:
small_ds = tiledbvcf.Dataset('data/small_dataset2', mode = "r")
small_ds.count()

## Remote VCFs

As of v0.5, `tiledbvcf` supports ingesting VCF files directly from remote locations, like AWS S3. The text file `data/s3-bcf-samples.txt` contains a list of S3 URIs pointing to 7 BCF files from the same cohort.

In [ ]:
with open("data/s3-bcf-samples.txt") as f:
    sample_uris = [l.rstrip("\n") for l in f.readlines()]

sample_uris

You can add them into your existing dataset by re-opening it in *write* mode and providing the file URIs. It's also necessary to allocate scratch space so the files can be downloaded to a temporary location prior to ingestion.

In [ ]:
small_ds = tiledbvcf.Dataset('data/small_dataset2', mode = "w")

small_ds.ingest_samples(
    sample_uris, 
    scratch_space_path = tempfile.gettempdir(), 
    scratch_space_size=10
)

The TileDB-VCF dataset located at `data/small_dataset2` now includes records for 660 variants across 10 samples. The next section provides examples demonstrating how to query this dataset.

*Hint: To facilitate large ingestions, TileDB-VCF also integrates with [AWS Batch](https://aws.amazon.com/batch/) and provides a [helper script](https://github.com/TileDB-Inc/TileDB-VCF/blob/master/apis/aws-batch/batch-ingest.py) to use this service for parallelizing ingestion across a user-defined number of batches.*

# Querying Variant Data

The `tiledbvcf.read()` function is your primary Python interface to the variant data stored in a TileDB-VCF dataset. It allows you to perform highly targeted queries and extract data for any subset of variants, samples, or attributes. To facilitate convenient downstream analysis, `tiledbvcf.read()` always returns results as a *pandas* `Data.Frame`.

Let's look at a small example focused on a few genes of interest. The genes are stored in a `dict` that indexes each gene's genomic range.

In [ ]:
genes = {
    "ZNF595": "4:53227-196092",
    "DOCK8":  "9:214865-465259",
    "DIP2C":  "10:320130-735608",
    "RPH3AL": "17:62180-202633"
}

First, make sure you have opened the dataset in *read* mode.

In [ ]:
small_ds = tiledbvcf.Dataset('data/small_dataset2', mode = "r")

Here, we query our dataset for any variants that overlap *ZNF595* and specify which attributes to include in the results `DataFrame`.

In [ ]:
small_ds.read(
  attrs = ['sample_name', 'pos_start'], 
  samples = ["G1", "G2"],
  regions = [genes.get("ZNF595")]
)

The `attrs` argument accepts the following values:
* `sample_name`
* `id` (variant identifier)
* `pos_start`
* `pos_end`
* `alleles`
* `filters`

Additionally, any individual `FORMAT` or `INFO` fields that were ingested into the dataset can be included by adding a `fmt_` or `info_` prefix, respectively. The TileDB-VCF schema uses these prefixes to avoid ambiguity in cases where an attribute like *DP* could come from the *FORMAT* or *INFO* field.

*Hint: See the [API reference](https://docs.tiledb.com/genomics/apis/python#read) for a complete list of queryable attributes.*

Let's expand this query to include a list of ranges so that all gene regions of interest are scanned for hits simultaneously. Additionally, search across *all* samples in the dataset, and include each variant's alleles and each sample's genotypes.

In [ ]:
samples = ["G" + str(i) for i in range(1,11)]

df = small_ds.read(
  attrs = ["sample_name", 'contig', 'pos_start', 'pos_end', 'alleles', 'fmt_GT'], 
  samples = samples,
  regions = list(genes.values())
)

df

The output format from `read()` is always a `DataFrame` in *"long"* format, in which each row represents an individual variant indexed by a specific sample and genomic position. However, you can easily reformat into a *"wide"* variant &times; sample genotype matrix using the `pivot()` method provided by *pandas*.

In [ ]:
df_wide = df.copy()
df_wide = df_wide.set_index(["contig", "pos_start"])
df_wide[["sample_name", "fmt_GT"]].pivot(columns = "sample_name")

At this point, you've created and queried a small example dataset. Next you will work with a more realistically sized dataset and utilize `tiledfvcf`'s features for handling large queries, which can enable incremental/out-of-core processing in the event that the requested data is too large to fit in main memory.

# Handling Large Queries

Unlike the CLI, which exports directly to disk, results for queries performed using Python are read into memory. Therefore, when querying even moderately sized genomic datasets, the amount of available memory must be taken into consideration. 

Here, we will demonstrate several of `tilebvcf`'s features for overcoming memory limitations when querying large datasets. To do so, the following examples will utilize a new example TileDB-VCF dataset that contains 20 samples with over 20 million variants each. This dataset is publicly available on S3 and TileDB Cloud, and can be access with `tiledbvcf` by simply providing the appropriate URI.

In [ ]:
import tiledbvcf
uri = "data/vcf-samples-20"

# uri = "tiledb://vcf-samples-20"
# uri = "s3://tiledb-inc-demo-data/tiledb-arrays/2.0/vcf-samples-20"

## Memory Budget

One strategy for accommodating large queries is to simply increase the amount of memory available to `tiledbvcf`. By default `tiledbvcf` allocates 2Gb of memory for queries. However, this value can be adjusted using the `memory_budget_mb` parameter. For the purposes of this tutorial the budget will be *decreased* to demonstrate how `tiledbvcf` is able to perform genome-scale queries even in a memory constrained environment.

In [ ]:
cfg = tiledbvcf.ReadConfig(memory_budget_mb=256)
ds = tiledbvcf.Dataset(uri, mode = "r", cfg = cfg)

You can read more about how TileDB-VCF allocates memory [here](https://docs.tiledb.com/genomics/advanced/read-algorithm#buffer-allocation).

## Batched Reads

For queries that encompass many genomic regions you can simply provide an external `bed` file. In this example, you will query for any variants located in the promoter region of a known gene located on chromosomes 1–4. 

After performing a query, you can use `read_completed()` to verify whether or not *all* results were succesfully returned.

In [ ]:
attrs = ["sample_name", "contig", "pos_start", "fmt_GT"]
df = ds.read(attrs, bed_file = "data/gene-promoters-hg38.bed")
ds.read_completed()

In this case, it returned `False`, indicating the requested data was too large to fit in memory so `tiledbvcf` retrieved as many records as possible in this first batch. The reamining records can be retrieved using `continue_read()`. Here, we've setup our code to accomodate the possibility that the full set of results are split across multiple batches.

In [ ]:
print ("The dataframe contains")

while not ds.read_completed():
    print (f"\t...{df.shape[0]} rows")
    df = df.append(ds.continue_read())

print (f"\t...{df.shape[0]} rows")

## Iteration

A Python generator version of read function is also provided. This pattern provides a powerful interface for batch processing variant data.

In [ ]:
ds = tiledbvcf.Dataset(uri, mode = "r", cfg = cfg)

attrs.append("query_bed_start")

df = pd.DataFrame()
for batch in ds.read_iter(attrs, bed_file = "data/gene-promoters-hg38.bed"):
    df = df.append(batch, ignore_index = True)
    
df